In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import multiprocessing
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer


import tensorflow_hub as hub

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

In [2]:
df = pd.read_csv('train.csv', header = None, usecols = [1,2], nrows = None, encoding = 'ISO-8859-1')

In [3]:
import re

pattern= r'(\w+|\!+|\.+|\,+|\?+)'
def findmatches(sentences):
    output = []
    for sen in sentences:
        match= re.findall(pattern, sen)
        cleaned = []
        for word in match:
            if word.find('.')>-1 :
                
                cleaned.append('.')
            elif word.find(',')>-1  :
                cleaned.append(',')
            elif word.find('?')>-1  :
                cleaned.append('?')
            elif word.find('!')>-1  :
                cleaned.append('!')
            else:
                cleaned.append(word)

        output.append(cleaned)
    return output

sentences = findmatches( df.values[1:][:,1])

In [4]:
def GetWords(sentences) :

    words = []
    for tokens in sentences:
        for word in tokens:
            words.append(word)

    words = set(words)

    words = sorted(words)

    AllWords = list(words)

    return AllWords

Words = GetWords(sentences)

In [6]:
Labels = [int(a) for a in df.values[1:][:,0]]

In [7]:
import random
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import copy
from datetime import datetime
np.random.seed(1)

def preprocessing(input):
    newInput = []
    maxLen = max([len(l) for l in input])
    for sentence in input:
        currEmbedding = copy.deepcopy(sentence)
        curLen = len(sentence)
        if(curLen < maxLen) :
            newSentence = np.zeros(len(sentence[0]))
            for i in range(curLen,maxLen) :
                currEmbedding.append(newSentence)
        newInput.append(currEmbedding)
    return newInput

def LSTMModel(vectorLength):
    #embedding = Input()
    net_input = Input(shape=(None,vectorLength))
    X = LSTM(128, return_sequences=True)(net_input)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(1, activation=None)(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=net_input, outputs=X)

    return model;

Using TensorFlow backend.


In [8]:

def generateInput(sents,labs,batch_size,LenOfEmbedding,maxL,AllEmbeddings):
    while True:
        combined = copy.deepcopy(list(zip(sents,labs)))
        random.seed(datetime.now())
        random.shuffle(combined)
        column = np.zeros(LenOfEmbedding)
        (sentences, labels) = list(zip(*combined))

        X = []
        for sen in sentences[:batch_size]:
            embed = []
            for word in sen:
                embed.append(AllEmbeddings[word])
            for i in range(len(embed),maxL):
                embed.append(column)
            X.append(embed)
        Y = [y for y in labels[:batch_size]]
        yield(np.asarray(X), np.asarray(Y))

def generateOutput(sents,labs,batch_size,LenOfEmbedding,maxL,AllEmbeddings):
    i = 0
    while True:
        combined = copy.deepcopy(list(zip(sents,labs)))
        (sentences, labels) = list(zip(*combined))
        column = np.zeros(LenOfEmbedding)
        X = []
        for sen in sentences[:batch_size]:
            embed = []
            for word in sen:
                embed.append(AllEmbeddings[word])
            for i in range(len(embed),maxL):
                embed.append(column)
            X.append(embed)
        Y = [y for y in labels[:batch_size]]
        i=i+1
        yield(np.asarray(X), np.asarray(Y))


partition = 100
nrows = 200
epoch = 100
batch_size = 10

model = LSTMModel(LenOfEmbedding)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

maxL = max([len(s) for s in sentences])

model.fit_generator(generateInput(sentences[:partition],Labels[:partition],batch_size,LenOfEmbedding,maxL,AllEmbeddings),int(partition/batch_size), epochs = epoch)

loss, acc, f1_score, precision, recall = model.evaluate_generator(generateOutput(sentences[partition:],Labels[partition:],len(Labels[partition:]),LenOfEmbedding,maxL,AllEmbeddings),1)

print("Test accuracy = ", acc)


Epoch 1/100
10/10 [==============================] - 78s 8s/step - loss: 0.6639 - acc: 0.7500
Epoch 2/100
10/10 [==============================] - 37s 4s/step - loss: 0.5565 - acc: 0.7800
Epoch 3/100
10/10 [==============================] - 36s 4s/step - loss: 0.5579 - acc: 0.7500
Epoch 4/100
10/10 [==============================] - 37s 4s/step - loss: 0.6044 - acc: 0.7500
Epoch 5/100
10/10 [==============================] - 38s 4s/step - loss: 0.5488 - acc: 0.7900
Epoch 6/100
10/10 [==============================] - 35s 4s/step - loss: 0.5438 - acc: 0.7800
Epoch 7/100
10/10 [==============================] - 37s 4s/step - loss: 0.5730 - acc: 0.7400
Epoch 8/100
10/10 [==============================] - 37s 4s/step - loss: 0.5158 - acc: 0.8100
Epoch 9/100
10/10 [==============================] - 37s 4s/step - loss: 0.5576 - acc: 0.7500
Epoch 10/100
10/10 [==============================] - 37s 4s/step - loss: 0.5549 - acc: 0.7600
Epoch 11/100
10/10 [==============================] - 37s 4

MemoryError: 